<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Lab-01:-Vanilla-RNN---demo" data-toc-modified-id="Lab-01:-Vanilla-RNN---demo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lab 01: Vanilla RNN - demo</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#With-or-without-GPU?" data-toc-modified-id="With-or-without-GPU?-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>With or without GPU?</a></span></li><li><span><a href="#Download-Penn-Tree-Bank" data-toc-modified-id="Download-Penn-Tree-Bank-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Download Penn Tree Bank</a></span></li><li><span><a href="#Some-constants-associated-with-the-data-set" data-toc-modified-id="Some-constants-associated-with-the-data-set-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Some constants associated with the data set</a></span></li><li><span><a href="#Make-a-recurrent-net-class" data-toc-modified-id="Make-a-recurrent-net-class-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Make a recurrent net class</a></span></li><li><span><a href="#Build-the-net.-Choose-the-hidden-size-to-be-150.-How-many-parameters-in-total?" data-toc-modified-id="Build-the-net.-Choose-the-hidden-size-to-be-150.-How-many-parameters-in-total?-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Build the net. Choose the hidden size to be 150. How many parameters in total?</a></span></li><li><span><a href="#Send-the-weights-of-the-networks-to-the-GPU" data-toc-modified-id="Send-the-weights-of-the-networks-to-the-GPU-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>Send the weights of the networks to the GPU</a></span></li><li><span><a href="#Set-up-manually-the-weights-of-the-embedding-module-and-Linear-module" data-toc-modified-id="Set-up-manually-the-weights-of-the-embedding-module-and-Linear-module-1.0.7"><span class="toc-item-num">1.0.7&nbsp;&nbsp;</span>Set up manually the weights of the embedding module and Linear module</a></span></li><li><span><a href="#Choose-the-criterion,-as-well-as-the-following-important-hyperparameters:" data-toc-modified-id="Choose-the-criterion,-as-well-as-the-following-important-hyperparameters:-1.0.8"><span class="toc-item-num">1.0.8&nbsp;&nbsp;</span>Choose the criterion, as well as the following important hyperparameters:</a></span></li><li><span><a href="#Function-to-evaluate-the-network-on-the-test-set" data-toc-modified-id="Function-to-evaluate-the-network-on-the-test-set-1.0.9"><span class="toc-item-num">1.0.9&nbsp;&nbsp;</span>Function to evaluate the network on the test set</a></span></li><li><span><a href="#Do-10-passes-through-the-training-set-(100-passes-would-reach-135-on-test-set)" data-toc-modified-id="Do-10-passes-through-the-training-set-(100-passes-would-reach-135-on-test-set)-1.0.10"><span class="toc-item-num">1.0.10&nbsp;&nbsp;</span>Do 10 passes through the training set (100 passes would reach 135 on test set)</a></span></li><li><span><a href="#Choose-one-sentence-(taken-from-the-test-set)" data-toc-modified-id="Choose-one-sentence-(taken-from-the-test-set)-1.0.11"><span class="toc-item-num">1.0.11&nbsp;&nbsp;</span>Choose one sentence (taken from the test set)</a></span></li><li><span><a href="#Convert-the-sentence-into-a-vector,-then-send-to-GPU" data-toc-modified-id="Convert-the-sentence-into-a-vector,-then-send-to-GPU-1.0.12"><span class="toc-item-num">1.0.12&nbsp;&nbsp;</span>Convert the sentence into a vector, then send to GPU</a></span></li><li><span><a href="#Set-the-initial-hidden-state-to-zero,-then-run-the-RNN." data-toc-modified-id="Set-the-initial-hidden-state-to-zero,-then-run-the-RNN.-1.0.13"><span class="toc-item-num">1.0.13&nbsp;&nbsp;</span>Set the initial hidden state to zero, then run the RNN.</a></span></li><li><span><a href="#Display-the-network-prediction-for-the-next-word" data-toc-modified-id="Display-the-network-prediction-for-the-next-word-1.0.14"><span class="toc-item-num">1.0.14&nbsp;&nbsp;</span>Display the network prediction for the next word</a></span></li></ul></li></ul></li></ul></div>

# Lab 01: Vanilla RNN - demo

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'vrnn_demo.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    print(path_to_file)
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    os.chdir(path_to_file)
    !pwd

In [1]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 153 sec ( 2.55 min)<br> 
* Time for 1 epoch on GPU : 8.4 sec w/ GeForce GTX 1080 Ti <br>

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device, torch.cuda.get_device_name(device=None))

cuda GeForce GTX TITAN X


In [4]:
torch.cuda.empty_cache()

### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [5]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'librivox_fr/train_data.pt')
test_data   =  torch.load(data_path+'librivox_fr/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([20542, 20])
torch.Size([74, 20])


In [6]:
seq_length = 35
for count in range( 0 , 20542-seq_length ,  seq_length):

    # create a minibatch
    minibatch_data =  train_data[ count   : count+seq_length   ]
    print(minibatch_data.shape)
    print(minibatch_data[:2])
    break

torch.Size([35, 20])
tensor([[   1,   28,  170,  640,  446,  210,  138, 1734,  365,   20,   20, 2885,
         2678,  193,  773,   25,  377,  328, 3061, 4869],
        [   2,  457,   43,   13,  122,  256,    5, 4486,   36,   23,   34,    0,
          124, 1956,  235, 4466,    0, 7351,  369,    0]])


In [7]:
max(torch.unique(train_data)), len((torch.unique(train_data)))

(tensor(9573), 9574)

In [8]:
max(torch.unique(test_data)), len((torch.unique(test_data)))

(tensor(9575), 503)

In [ ]:
test_data

### Some constants associated with the data set

In [9]:
bs = 20 # if you change this, also change this in generate_librivox_fr
# bs = 10

# vocab_size = 17498 # if WORD_OCC_THRESHOLD == 1
vocab_size = 9574 # if WORD_OCC_THRESHOLD == 3

### Make a recurrent net class

In [10]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.RNN(       hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1( word_seq )  
        h_seq , h_final     =   self.layer2( g_seq , h_init )
        score_seq           =   self.layer3( h_seq )
        
        return score_seq,  h_final 


### Build the net. Choose the hidden size to be 150. How many parameters in total?

In [11]:
hidden_size=50

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(9574, 50)
  (layer2): RNN(50, 50)
  (layer3): Linear(in_features=50, out_features=9574, bias=True)
)
There are 972074 (0.97 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [12]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [13]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 1
* sequence length = 35

In [14]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

### Function to evaluate the network on the test set

In [ ]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    
    h=h.to(device)

       
    for count in range( 0 , 74-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h  = net( minibatch_data, h )
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 10 passes through the training set (100 passes would reach 135 on test set)

In [15]:
start=time.time()

for epoch in range(10):
    
    # keep the learning rate to 1 during the first 4 epochs, then divide by 1.1 at every epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # create a new optimizer and give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(1, bs, hidden_size)

    # send it to the gpu    
    h=h.to(device)
    
    for count in range( 0 , 20542-seq_length ,  seq_length):
             
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        h=h.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h  = net( minibatch_data, h )
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
#     eval_on_test_set() 



epoch= 0 	 time= 2.5484092235565186 	 lr= 1 	 exp(loss)= 683.3492200973142

epoch= 1 	 time= 5.117347717285156 	 lr= 1 	 exp(loss)= 413.63328776051367

epoch= 2 	 time= 7.706781625747681 	 lr= 1 	 exp(loss)= 318.7099496793693

epoch= 3 	 time= 10.250564098358154 	 lr= 1 	 exp(loss)= 265.82340066046135

epoch= 4 	 time= 12.89343786239624 	 lr= 0.9090909090909091 	 exp(loss)= 232.74181976756424

epoch= 5 	 time= 15.543511867523193 	 lr= 0.8264462809917354 	 exp(loss)= 211.59802737378558

epoch= 6 	 time= 18.28880524635315 	 lr= 0.7513148009015777 	 exp(loss)= 196.66495896463357

epoch= 7 	 time= 20.686213731765747 	 lr= 0.6830134553650705 	 exp(loss)= 185.55740052879185

epoch= 8 	 time= 23.173760652542114 	 lr= 0.6209213230591549 	 exp(loss)= 176.86018173652417

epoch= 9 	 time= 25.88524031639099 	 lr= 0.5644739300537771 	 exp(loss)= 169.8470620771973


### Choose one sentence (taken from the test set)

In [77]:
sentence1 = "on entendait vaguement au dehors les"

sentence2 = "hier je luttai de la sorte contre le grand"

sentence3 = "il connaissait la route et nous avons"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# or make your own sentence.  No capital letter or punctuation allowed. Each word must be in the allowed vocabulary.
sentence6= "he was very"

# SELECT THE SENTENCE HERE
mysentence = sentence1

### Convert the sentence into a vector, then send to GPU

In [71]:
minibatch_data=utils.sentence2vector_librivox_fr(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[ 370],
        [3291],
        [  23],
        [ 636],
        [   8],
        [ 457],
        [ 695]], device='cuda:0')


### Set the initial hidden state to zero, then run the RNN.

In [40]:
h = torch.zeros(1, 1, hidden_size)
h=h.to(device)

scores , h = net( minibatch_data , h )

### Display the network prediction for the next word

In [35]:
print(mysentence, '... \n')

utils.show_next_word(scores)

il connaissait la route et nous avons ... 

6.6%	 <unk>
4.1%	 fait
3.5%	 de
3.4%	 été
2.8%	 à
2.3%	 eu
2.1%	 dit
1.9%	 en
1.8%	 vu
1.7%	 plus
1.5%	 bien
1.5%	 des
1.4%	 un
1.1%	 la
1.0%	 les
1.0%	 encore
0.9%	 donné
0.9%	 une
0.8%	 toujours
0.8%	 pris
0.8%	 pour
0.8%	 que
0.7%	 le
0.7%	 dans
0.7%	 pu
0.7%	 mis
0.6%	 donc
0.6%	 raison
0.6%	 déjà
0.6%	 <eos>


In [56]:
path_data = '../'
word2idx  =  torch.load(path_data + 'librivox_fr/word2idx.pt')
idx2word  =  torch.load(path_data + 'librivox_fr/idx2word.pt')


In [60]:
def get_next_word(scores):
    prob=F.softmax(scores,dim=2)
    num_word_display = 2
    p=prob[-1].squeeze()
    p, word_idx = torch.topk(p, num_word_display)
#     print(p, word_idx)
    if word_idx[0] == 0:
        return idx2word[word_idx[1]]
    else:
        return idx2word[word_idx[0]]
    

In [58]:
next_word = get_next_word(scores)

tensor([0.0664, 0.0412], device='cuda:0', grad_fn=<TopkBackward>) tensor([  0, 503], device='cuda:0')


'fait'

In [78]:
print(mysentence + '...')

i= 0
not_finished = True
while i < 10 and not_finished :

    minibatch_data=utils.sentence2vector_librivox_fr(mysentence)
      
    minibatch_data=minibatch_data.to(device)
    h = torch.zeros(1, 1, hidden_size)
    h=h.to(device)
    scores , h = net( minibatch_data , h )
    
    next_word = get_next_word(scores)
    mysentence += ' ' + next_word
    print(mysentence + '...')
    not_finished = next_word != '<eos>'
    i += 1
    

on entendait vaguement au dehors les...
on entendait vaguement au dehors les autres...
on entendait vaguement au dehors les autres et...
on entendait vaguement au dehors les autres et les...
on entendait vaguement au dehors les autres et les autres...
on entendait vaguement au dehors les autres et les autres et...
on entendait vaguement au dehors les autres et les autres et les...
on entendait vaguement au dehors les autres et les autres et les autres...
on entendait vaguement au dehors les autres et les autres et les autres et...
on entendait vaguement au dehors les autres et les autres et les autres et les...
on entendait vaguement au dehors les autres et les autres et les autres et les autres...
